In [1]:
import numpy as np
import pandas as pd
from boruta import BorutaPy
from sklearn.datasets import load_diabetes
from sklearn.metrics import accuracy_score 
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split

In [9]:
%cd S:\PPMI_RNA\Processed\Female
data = pd.read_csv('All_Counts_PatAndHealthy_Disease_Female.csv')
data['DIAGNOSIS'] = data['DIAGNOSIS'].replace({'PD': 1, 'Control': 0})
data.head(1)

S:\PPMI_RNA\Processed\Female


,HudAlphaID,DIAGNOSIS,PATNO,ENSG00000223972.5,ENSG00000227232.5,ENSG00000278267.1,ENSG00000243485.5,ENSG00000284332.1,ENSG00000237613.2,ENSG00000268020.3,...,ENSG00000273739.1,ENSG00000276700.1,ENSG00000276312.1,ENSG00000275757.1,ENSG00000278573.1,ENSG00000276017.1,ENSG00000278817.1,ENSG00000277196.4,ENSG00000278625.1,ENSG00000277374.1
0,5104_SL_0002,0,3201,2,199,10,0,0,0,0,...,0,0,2,0,0,0,3,0,0,0


In [10]:
X = data.drop(['HudAlphaID','PATNO'], axis = 1)
y = data['DIAGNOSIS']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = .2, random_state = 1) 
print(X.head(2))

   DIAGNOSIS  ENSG00000223972.5  ENSG00000227232.5  ENSG00000278267.1  \
0          0                  2                199                 10   
1          0                  0                148                  2   

   ENSG00000243485.5  ENSG00000284332.1  ENSG00000237613.2  ENSG00000268020.3  \
0                  0                  0                  0                  0   
1                  1                  0                  1                  0   

   ENSG00000240361.2  ENSG00000186092.6  ...  ENSG00000273739.1  \
0                  1                  0  ...                  0   
1                  1                  0  ...                  0   

   ENSG00000276700.1  ENSG00000276312.1  ENSG00000275757.1  ENSG00000278573.1  \
0                  0                  2                  0                  0   
1                  3                  8                  2                  0   

   ENSG00000276017.1  ENSG00000278817.1  ENSG00000277196.4  ENSG00000278625.1  \
0        

In [11]:
rf_all_features = RandomForestClassifier(random_state=1)
rf_all_features.fit(X_train, y_train)

RandomForestClassifier(random_state=1)

In [12]:
accuracy_score(y_test, rf_all_features.predict(X_test))

0.7254901960784313

In [13]:
rfc = RandomForestClassifier(random_state=1)
boruta_selector = BorutaPy(rfc, n_estimators= 'auto', verbose=1, random_state=1)
boruta_selector.fit(np.array(X_train), np.array(y_train))  

Iteration: 1 / 100
Iteration: 2 / 100
Iteration: 3 / 100
Iteration: 4 / 100
Iteration: 5 / 100
Iteration: 6 / 100
Iteration: 7 / 100
Iteration: 8 / 100
Iteration: 9 / 100
Iteration: 10 / 100
Iteration: 11 / 100
Iteration: 12 / 100
Iteration: 13 / 100
Iteration: 14 / 100
Iteration: 15 / 100
Iteration: 16 / 100
Iteration: 17 / 100
Iteration: 18 / 100
Iteration: 19 / 100
Iteration: 20 / 100
Iteration: 21 / 100
Iteration: 22 / 100
Iteration: 23 / 100
Iteration: 24 / 100
Iteration: 25 / 100
Iteration: 26 / 100
Iteration: 27 / 100
Iteration: 28 / 100
Iteration: 29 / 100
Iteration: 30 / 100
Iteration: 31 / 100
Iteration: 32 / 100
Iteration: 33 / 100
Iteration: 34 / 100
Iteration: 35 / 100
Iteration: 36 / 100
Iteration: 37 / 100
Iteration: 38 / 100
Iteration: 39 / 100
Iteration: 40 / 100
Iteration: 41 / 100
Iteration: 42 / 100
Iteration: 43 / 100
Iteration: 44 / 100
Iteration: 45 / 100
Iteration: 46 / 100
Iteration: 47 / 100
Iteration: 48 / 100
Iteration: 49 / 100
Iteration: 50 / 100
Iteration

BorutaPy(estimator=RandomForestClassifier(n_estimators=102,
                                          random_state=RandomState(MT19937) at 0x1AD0F6EE840),
         n_estimators='auto',
         random_state=RandomState(MT19937) at 0x1AD0F6EE840, verbose=1)

In [14]:
# print("Ranking: ",boruta_selector.ranking_)          
print("No. of significant features: ", boruta_selector.n_features_)

No. of significant features:  35


In [15]:
selected_rf_features = pd.DataFrame({'CpG':list(X_train.columns),
                                       'Ranking':boruta_selector.ranking_})
selected_rf_features = selected_rf_features.sort_values(by='Ranking')

In [16]:
Top50 = selected_rf_features.iloc[:50, :]
Top50.head(3)

,CpG,Ranking
0,DIAGNOSIS,1
29028,ENSG00000204177.10,1
20046,ENSG00000228495.1,1


### Top 50

In [17]:
Top50.to_csv ('DEGs_Boruta_Top50_Female.csv', index = False)
selected_rf_features.to_csv ('DEGs_Boruta_All_Female.csv', index = False)

In [18]:
# create a list of column names to subset from data
col_names = Top50.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['DIAGNOSIS'] = data['DIAGNOSIS']
df_subset.to_csv ('DEGs_Boruta_Top50_Data_Female.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_9152\1564645157.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['DIAGNOSIS'] = data['DIAGNOSIS']


,DIAGNOSIS,ENSG00000188822.7,ENSG00000156869.13,ENSG00000183508.4,ENSG00000225217.1,ENSG00000180667.10,ENSG00000116574.5,ENSG00000134317.17,ENSG00000179915.22,ENSG00000091181.19,...,ENSG00000268240.1,ENSG00000131401.11,ENSG00000130433.7,ENSG00000205927.4,ENSG00000182240.15,ENSG00000224388.1,ENSG00000182362.13,ENSG00000100079.6,ENSG00000234493.4,ENSG00000278384.1
0,0,500,323,28981,3330,24875,1191,191,305,782,...,95,1318,20,56,497,38,343,969,27,124
1,0,354,368,77677,1148,27435,815,117,118,1242,...,94,908,72,91,444,42,288,423,39,16


### Top 100

In [19]:
Top100 = selected_rf_features.iloc[:100, :]
Top100.to_csv ('DEGs_Boruta_Top100_Female.csv', index = False)
Top100.head(2)

,CpG,Ranking
0,DIAGNOSIS,1
29028,ENSG00000204177.10,1


In [20]:
# create a list of column names to subset from data
col_names = Top100.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['DIAGNOSIS'] = data['DIAGNOSIS']
df_subset.to_csv ('DEGs_Boruta_Top100_Data_Female.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_9152\3584030340.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['DIAGNOSIS'] = data['DIAGNOSIS']


,DIAGNOSIS,ENSG00000271806.1,ENSG00000057757.9,ENSG00000188822.7,ENSG00000156869.13,ENSG00000282608.1,ENSG00000134256.12,ENSG00000183508.4,ENSG00000163554.13,ENSG00000225217.1,...,ENSG00000189068.10,ENSG00000205927.4,ENSG00000184221.12,ENSG00000182240.15,ENSG00000224388.1,ENSG00000182362.13,ENSG00000100079.6,ENSG00000047597.6,ENSG00000234493.4,ENSG00000278384.1
0,0,37,7505,500,323,70,848,28981,430,3330,...,366,56,118,497,38,343,969,3354,27,124
1,0,26,10504,354,368,62,714,77677,1162,1148,...,209,91,104,444,42,288,423,6188,39,16


### Top 250

In [21]:
Top250 = selected_rf_features.iloc[:250, :]
Top250.to_csv ('DEGs_Boruta_Top250_Female.csv', index = False)
Top250.head(2)

,CpG,Ranking
0,DIAGNOSIS,1
29028,ENSG00000204177.10,1


In [22]:
# create a list of column names to subset from data
col_names = Top250.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['DIAGNOSIS'] = data['DIAGNOSIS']
df_subset.to_csv ('DEGs_Boruta_Top250_Data_Female.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_9152\916652613.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['DIAGNOSIS'] = data['DIAGNOSIS']


,DIAGNOSIS,ENSG00000271806.1,ENSG00000117115.12,ENSG00000162543.5,ENSG00000076864.19,ENSG00000057757.9,ENSG00000188822.7,ENSG00000116983.12,ENSG00000117461.14,ENSG00000162437.14,...,ENSG00000100079.6,ENSG00000073150.13,ENSG00000047597.6,ENSG00000238222.3,ENSG00000231725.1,ENSG00000234493.4,ENSG00000076770.14,ENSG00000233093.5,ENSG00000210082.2,ENSG00000278384.1
0,0,37,1745,208,27,7505,500,351,114,217,...,969,84,3354,5,18,27,35787,79,181,124
1,0,26,2239,369,46,10504,354,178,80,214,...,423,75,6188,9,3,39,34380,71,653,16


### Top 500

In [23]:
Top500 = selected_rf_features.iloc[:250, :]
Top500.to_csv ('DEGs_Boruta_Top500_Female.csv', index = False)
Top500.head(2)

,CpG,Ranking
0,DIAGNOSIS,1
29028,ENSG00000204177.10,1


In [24]:
# create a list of column names to subset from data
col_names = Top500.iloc[:, 0].tolist()

# subset df based on the column names
df_subset = data.loc[:, data.columns.isin(col_names)]
df_subset['DIAGNOSIS'] = data['DIAGNOSIS']
df_subset.to_csv ('DEGs_Boruta_Top500_Data_Female.csv', index = False)
df_subset.head(2)

C:\Users\system 4\AppData\Local\Temp\ipykernel_9152\2524728834.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset['DIAGNOSIS'] = data['DIAGNOSIS']


,DIAGNOSIS,ENSG00000271806.1,ENSG00000117115.12,ENSG00000162543.5,ENSG00000076864.19,ENSG00000057757.9,ENSG00000188822.7,ENSG00000116983.12,ENSG00000117461.14,ENSG00000162437.14,...,ENSG00000100079.6,ENSG00000073150.13,ENSG00000047597.6,ENSG00000238222.3,ENSG00000231725.1,ENSG00000234493.4,ENSG00000076770.14,ENSG00000233093.5,ENSG00000210082.2,ENSG00000278384.1
0,0,37,1745,208,27,7505,500,351,114,217,...,969,84,3354,5,18,27,35787,79,181,124
1,0,26,2239,369,46,10504,354,178,80,214,...,423,75,6188,9,3,39,34380,71,653,16
